In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from pathlib import Path

In [5]:
df = pd.read_pickle("Asthma_Annotations.pkl")
df.head()

,id,image_id,image_name,image_set,class,vector,unique_identifier,user_id,deleted,last_editor
0,172441,221,BAL 1 Spray 2.svs,14,Makrophagen,"{'x1': 13612, 'x2': 13666, 'y1': 12371, 'y2': ...",a268b46f-e13f-480e-b96d-c7900790ad5f,10,False,None
1,172442,221,BAL 1 Spray 2.svs,14,Makrophagen,"{'x1': 13562, 'x2': 13618, 'y1': 12437, 'y2': ...",e8d67efd-0baf-4f78-ad73-b8f77422b826,10,False,None
2,172443,221,BAL 1 Spray 2.svs,14,Makrophagen,"{'x1': 13482, 'x2': 13546, 'y1': 12458, 'y2': ...",55aa3d51-7f30-43a0-83ef-d9acc74d9a21,10,False,None
3,172444,221,BAL 1 Spray 2.svs,14,Makrophagen,"{'x1': 13471, 'x2': 13533, 'y1': 12281, 'y2': ...",8f259e8f-119b-44de-8a90-2c53e7ee0e60,10,False,None
4,172446,221,BAL 1 Spray 2.svs,14,Makrophagen,"{'x1': 13591, 'x2': 13673, 'y1': 12204, 'y2': ...",052b3f5a-f6f6-4b0d-a134-9f4797601dc4,10,False,None


In [6]:
df["image_name"].unique()

array(['BAL 1 Spray 2.svs', 'BAL Promyk Spray 4.svs',
       'BAL AIA Blickfang Luft.svs', 'BAL Booker Spray 3.svs',
       'BAL Bubi Spray 1.svs', 'BAL cent blue Luft 2.svs'], dtype=object)

In [7]:
df["class"].unique()

array(['Makrophagen', 'Lymohozyten', 'Neutrophile', 'Mastzellen',
       'Eosinophile', 'Mehrkernige', 'Erythrozyt', 'Weitere', 'Schrott',
       'ROI'], dtype=object)

In [10]:
df = df[df["class"].isin(["Eosinophile", "Mastzellen", "Neutrophile", "Makrophagen", "Lymohozyten"])]
df.head()

,id,image_id,image_name,image_set,class,vector,unique_identifier,user_id,deleted,last_editor
0,172441,221,BAL 1 Spray 2.svs,14,Makrophagen,"{'x1': 13612, 'x2': 13666, 'y1': 12371, 'y2': ...",a268b46f-e13f-480e-b96d-c7900790ad5f,10,False,None
1,172442,221,BAL 1 Spray 2.svs,14,Makrophagen,"{'x1': 13562, 'x2': 13618, 'y1': 12437, 'y2': ...",e8d67efd-0baf-4f78-ad73-b8f77422b826,10,False,None
2,172443,221,BAL 1 Spray 2.svs,14,Makrophagen,"{'x1': 13482, 'x2': 13546, 'y1': 12458, 'y2': ...",55aa3d51-7f30-43a0-83ef-d9acc74d9a21,10,False,None
3,172444,221,BAL 1 Spray 2.svs,14,Makrophagen,"{'x1': 13471, 'x2': 13533, 'y1': 12281, 'y2': ...",8f259e8f-119b-44de-8a90-2c53e7ee0e60,10,False,None
4,172446,221,BAL 1 Spray 2.svs,14,Makrophagen,"{'x1': 13591, 'x2': 13673, 'y1': 12204, 'y2': ...",052b3f5a-f6f6-4b0d-a134-9f4797601dc4,10,False,None


In [19]:
data = []
total = {"Eosinophile":0, "Mastzellen":0, "Neutrophile":0, "Makrophagen":0, "Lymohozyten":0}
row_totals = 0
for image in df["image_name"].unique():
    imageDF = df[df["image_name"] == image]
    
    row = []
    for label in ["Eosinophile", "Mastzellen", "Neutrophile", "Makrophagen", "Lymohozyten"]:

        num_label = imageDF[imageDF["class"] == label].shape[0]
        total[label] += num_label

        row.append(num_label)
        
    row_totals += sum(row)
    data.append([image] + row + [sum(row), 0])
        
data.append(["", total["Eosinophile"], total["Mastzellen"], total["Neutrophile"], total["Makrophagen"], total["Lymohozyten"], row_totals, 0])
dfStat = pd.DataFrame(data, columns=[ "file", "eosinophils", "mast cell", "neutrophils", "macrophages", "lymphocytes", "total", "screened"])
dfStat

,file,eosinophils,mast cell,neutrophils,macrophages,lymphocytes,total,screened
0,BAL 1 Spray 2.svs,10,69,1321,3081,15666,20147,0
1,BAL Promyk Spray 4.svs,21,511,3301,3934,14846,22613,0
2,BAL AIA Blickfang Luft.svs,47,762,951,16748,10342,28850,0
3,BAL Booker Spray 3.svs,20,37,2467,729,2144,5397,0
4,BAL Bubi Spray 1.svs,8,116,4491,1639,3077,9331,0
5,BAL cent blue Luft 2.svs,2,40,26,370,323,761,0
6,,108,1535,12557,26501,46398,87099,0


In [20]:
dfStat.to_csv(header=False, index=False, float_format='%.2f').replace(",","  &  ").replace("_","\_").replace("\r\n","\\ |").split(" |")

['BAL 1 Spray 2.svs  &  10  &  69  &  1321  &  3081  &  15666  &  20147  &  0\\',
 'BAL Promyk Spray 4.svs  &  21  &  511  &  3301  &  3934  &  14846  &  22613  &  0\\',
 'BAL AIA Blickfang Luft.svs  &  47  &  762  &  951  &  16748  &  10342  &  28850  &  0\\',
 'BAL Booker Spray 3.svs  &  20  &  37  &  2467  &  729  &  2144  &  5397  &  0\\',
 'BAL Bubi Spray 1.svs  &  8  &  116  &  4491  &  1639  &  3077  &  9331  &  0\\',
 'BAL cent blue Luft 2.svs  &  2  &  40  &  26  &  370  &  323  &  761  &  0\\',
 '  &  108  &  1535  &  12557  &  26501  &  46398  &  87099  &  0\\',
 '']